In [ ]:
!pip install -q bitsandbytes accelerate transformers datasets peft sentencepiece huggingface_hub streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
from collections import Counter
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from bert_score import BERTScorer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
class BiLSTM_GRU_Classifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, padding_idx=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
        self.bilstm    = nn.LSTM(embed_dim, hidden_dim,
                                 bidirectional=True, batch_first=True)
        self.gru       = nn.GRU(2*hidden_dim, hidden_dim, batch_first=True)
        self.classifier= nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        emb, _ = self.bilstm(self.embedding(x))
        out, _ = self.gru(emb)
        feat    = out.mean(dim=1)
        return self.classifier(feat)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/NLP/Datasets/lyrics_song_info.csv")
df = df.dropna(subset=["lyrics", "tags"])
df["primary_genre"] = df["tags"].str.split(";").str[0]

TOP_N = 100
top_genres = df["primary_genre"].value_counts().nlargest(TOP_N).index
df = df[df["primary_genre"].isin(top_genres)].reset_index(drop=True)

def tokenize(text):
    return re.findall(r"\w+'?\w*|[.,!?;]", text.lower())

counter = Counter()
for lyric in df["lyrics"]:
    counter.update(tokenize(lyric))
vocab = {tok: i+2 for i, (tok, _) in enumerate(counter.most_common(20_000))}
vocab["<pad>"] = 0
vocab["<unk>"] = 1

In [ ]:
labels = sorted(df["primary_genre"].unique())
label2id = {lab: i for i, lab in enumerate(labels)}
id2label = {i: lab for i, lab in enumerate(labels)}

print(f"Working with {len(labels)} music genres: {', '.join(labels)}")

Working with 100 music genres: Acoustic, Adult Alternative, Alternative, Alternative Metal, Alternative Pop, Alternative R&B, Alternative Rock, Art, Art Pop, Art Rock, Atlanta, Australia, Ballad, Baroque Pop, Beef, België/Belgique, Boom Bap, Boy Band, British Rock, Canada, Chicago Drill, Christian, Christian Rap, Cloud Rap, Conscious Hip-Hop, Country, Cover, DMV, Dance, Dance-Pop, Dancehall, Demo, Dirty South, Drill, East Coast, Eighties, Electro-Pop, Electronic, Electronic Rock, Emo, Emo Rap, Experimental, France, Freestyle, French Rap, Funk, G-Funk, Gangsta Rap, Glam Rock, Hard Rock, Hardcore Hip-Hop, Heavy Metal, History, Horrorcore, Indie, Indie Pop, Indie Rap, Indie Rock, Industrial Rock, Interlude, Ireland, Memes, Motown, Musicals, Neo Soul, Non-Music, Piano, Polska, Polska Muzyka, Polski Rap, Pop, Pop-Punk, Pop-Rock, Post-Hardcore, Power Pop, Producer, Progressive Rock, Psychedelic, Punk Rock, R&B, Rap, Rap Rock, Remix, Rock, Screen, Singer-Songwriter, Skit, Soul, Soul Pop, Soun

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bilstm_gru_model = BiLSTM_GRU_Classifier(
    vocab_size=len(vocab),
    embed_dim=128,
    hidden_dim=256,
    num_classes=len(labels)
).to(device)

In [ ]:
bilstm_gru_model.load_state_dict(torch.load("/content/drive/MyDrive/NLP/bilstm_gru_classifier_100.pt",
                                           map_location=device))
bilstm_gru_model.eval()

BiLSTM_GRU_Classifier(
  (embedding): Embedding(20002, 128, padding_idx=0)
  (bilstm): LSTM(128, 256, batch_first=True, bidirectional=True)
  (gru): GRU(512, 256, batch_first=True)
  (classifier): Linear(in_features=256, out_features=100, bias=True)
)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_name = "FacebookAI/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    df, test_size=0.1, stratify=df["primary_genre"], random_state=42
)

print(f"Train set: {len(train_df)} samples")
print(f"Validation set: {len(val_df)} samples")

Train set: 32055 samples
Validation set: 3562 samples


In [ ]:
def get_bilstm_predictions(model, texts):
    model.eval()

    predictions = []
    confidences = []

    for text in tqdm(texts, desc="Getting BiLSTM-GRU predictions"):
        tokens = tokenize(text)[:512]
        ids = [vocab.get(t, vocab["<unk>"]) for t in tokens]

        if len(ids) == 0:
            ids = [vocab["<unk>"]]

        input_tensor = torch.tensor([ids], dtype=torch.long).to(device)

        with torch.no_grad():
            logits = model(input_tensor)
            probs = torch.softmax(logits, dim=1)
            pred_idx = torch.argmax(probs, dim=1).item()
            confidence = probs[0][pred_idx].item()

        predictions.append(pred_idx)
        confidences.append(confidence)

    return predictions, confidences


In [ ]:
def prepare_dataset(df, include_predictions=True):
    if include_predictions:
        bilstm_preds, confidences = get_bilstm_predictions(bilstm_gru_model, df["lyrics"].tolist())

        dataset_dict = {
            "lyrics": df["lyrics"].tolist(),
            "original_genre": df["primary_genre"].tolist(),
            "original_label_id": [label2id[genre] for genre in df["primary_genre"]],
            "bilstm_prediction": [id2label[pred] for pred in bilstm_preds],
            "bilstm_pred_id": bilstm_preds,
            "confidence": confidences
        }
    else:
        dataset_dict = {
            "lyrics": df["lyrics"].tolist(),
            "original_genre": df["primary_genre"].tolist(),
            "original_label_id": [label2id[genre] for genre in df["primary_genre"]],
        }

    return Dataset.from_dict(dataset_dict)

In [ ]:
train_dataset = prepare_dataset(train_df, include_predictions=True)
val_dataset = prepare_dataset(val_df, include_predictions=True)

Getting BiLSTM-GRU predictions: 100%|██████████| 3562/3562 [00:49<00:00, 71.86it/s]


In [ ]:
def preprocess_function(examples):
    tokenized_inputs = tokenizer(
        examples["lyrics"],
        truncation=True,
        max_length=512,
        padding=False
    )

    tokenized_inputs["labels"] = examples["original_label_id"]

    return tokenized_inputs

In [ ]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/32055 [00:00<?, ? examples/s]

Map:   0%|          | 0/3562 [00:00<?, ? examples/s]

In [ ]:
import transformers
print(transformers.__version__)


4.51.3


In [ ]:
import inspect
from transformers import TrainingArguments
print(inspect.signature(TrainingArguments))


(output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType, str] = 'linear', lr_scheduler_kwargs: Union[dict, str, NoneType] = <factory>, warmup_ratio: float = 0.0, warmup_steps: in

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/NLP/RoBerta/roberta-lstm/results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    metric_for_best_model="accuracy",
    warmup_steps=500,
    logging_dir="/content/drive/MyDrive/NLP/RoBerta/roberta-lstm/logs",
    logging_steps=100,
)


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    accuracy = accuracy_score(labels, predictions)
    report = classification_report(labels, predictions, target_names=list(label2id.keys()), output_dict=True)

    results = {
        "accuracy": accuracy,
    }

    for genre, genre_id in label2id.items():
        if genre in report:
            results[f"{genre}_precision"] = report[genre]["precision"]
            results[f"{genre}_recall"] = report[genre]["recall"]
            results[f"{genre}_f1"] = report[genre]["f1-score"]

    results["macro_precision"] = report["macro avg"]["precision"]
    results["macro_recall"] = report["macro avg"]["recall"]
    results["macro_f1"] = report["macro avg"]["f1-score"]
    results["weighted_f1"] = report["weighted avg"]["f1-score"]

    return results

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback],
)

print("Starting model training...")
trainer.train()

<ipython-input-23-219c8ce38bc7>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Starting model training...


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: parames3 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Acoustic Precision,Acoustic Recall,Acoustic F1,Adult alternative Precision,Adult alternative Recall,Adult alternative F1,Alternative Precision,Alternative Recall,Alternative F1,Alternative metal Precision,Alternative metal Recall,Alternative metal F1,Alternative pop Precision,Alternative pop Recall,Alternative pop F1,Alternative r&b Precision,Alternative r&b Recall,Alternative r&b F1,Alternative rock Precision,Alternative rock Recall,Alternative rock F1,Art Precision,Art Recall,Art F1,Art pop Precision,Art pop Recall,Art pop F1,Art rock Precision,Art rock Recall,Art rock F1,Atlanta Precision,Atlanta Recall,Atlanta F1,Australia Precision,Australia Recall,Australia F1,Ballad Precision,Ballad Recall,Ballad F1,Baroque pop Precision,Baroque pop Recall,Baroque pop F1,Beef Precision,Beef Recall,Beef F1,België/belgique Precision,België/belgique Recall,België/belgique F1,Boom bap Precision,Boom bap Recall,Boom bap F1,Boy band Precision,Boy band Recall,Boy band F1,British rock Precision,British rock Recall,British rock F1,Canada Precision,Canada Recall,Canada F1,Chicago drill Precision,Chicago drill Recall,Chicago drill F1,Christian Precision,Christian Recall,Christian F1,Christian rap Precision,Christian rap Recall,Christian rap F1,Cloud rap Precision,Cloud rap Recall,Cloud rap F1,Conscious hip-hop Precision,Conscious hip-hop Recall,Conscious hip-hop F1,Country Precision,Country Recall,Country F1,Cover Precision,Cover Recall,Cover F1,Dmv Precision,Dmv Recall,Dmv F1,Dance Precision,Dance Recall,Dance F1,Dance-pop Precision,Dance-pop Recall,Dance-pop F1,Dancehall Precision,Dancehall Recall,Dancehall F1,Demo Precision,Demo Recall,Demo F1,Dirty south Precision,Dirty south Recall,Dirty south F1,Drill Precision,Drill Recall,Drill F1,East coast Precision,East coast Recall,East coast F1,Eighties Precision,Eighties Recall,Eighties F1,Electro-pop Precision,Electro-pop Recall,Electro-pop F1,Electronic Precision,Electronic Recall,Electronic F1,Electronic rock Precision,Electronic rock Recall,Electronic rock F1,Emo Precision,Emo Recall,Emo F1,Emo rap Precision,Emo rap Recall,Emo rap F1,Experimental Precision,Experimental Recall,Experimental F1,France Precision,France Recall,France F1,Freestyle Precision,Freestyle Recall,Freestyle F1,French rap Precision,French rap Recall,French rap F1,Funk Precision,Funk Recall,Funk F1,G-funk Precision,G-funk Recall,G-funk F1,Gangsta rap Precision,Gangsta rap Recall,Gangsta rap F1,Glam rock Precision,Glam rock Recall,Glam rock F1,Hard rock Precision,Hard rock Recall,Hard rock F1,Hardcore hip-hop Precision,Hardcore hip-hop Recall,Hardcore hip-hop F1,Heavy metal Precision,Heavy metal Recall,Heavy metal F1,History Precision,History Recall,History F1,Horrorcore Precision,Horrorcore Recall,Horrorcore F1,Indie Precision,Indie Recall,Indie F1,Indie pop Precision,Indie pop Recall,Indie pop F1,Indie rap Precision,Indie rap Recall,Indie rap F1,Indie rock Precision,Indie rock Recall,Indie rock F1,Industrial rock Precision,Industrial rock Recall,Industrial rock F1,Interlude Precision,Interlude Recall,Interlude F1,Ireland Precision,Ireland Recall,Ireland F1,Memes Precision,Memes Recall,Memes F1,Motown Precision,Motown Recall,Motown F1,Musicals Precision,Musicals Recall,Musicals F1,Neo soul Precision,Neo soul Recall,Neo soul F1,Non-music Precision,Non-music Recall,Non-music F1,Piano Precision,Piano Recall,Piano F1,Polska Precision,Polska Recall,Polska F1,Polska muzyka Precision,Polska muzyka Recall,Polska muzyka F1,Polski rap Precision,Polski rap Recall,Polski rap F1,Pop Precision,Pop Recall,Pop F1,Pop-punk Precision,Pop-punk Recall,Pop-punk F1,Pop-rock Precision,Pop-rock Recall,Pop-rock F1,Post-hardcore Precision,Post-hardcore Recall,Post-hardcore F1,Power pop Precision,Power pop Recall,Power pop F1,Producer Precision,Producer Recall,Producer F1,Progressive rock Precision,Progressive rock Recall,Progressive rock F1,Psychedelic Precision,Psychedelic Recall,Psychedelic F1,Punk rock Pre

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

TrainOutput(global_step=10020, training_loss=1.7786762446938398, metrics={'train_runtime': 3330.8747, 'train_samples_per_second': 48.118, 'train_steps_per_second': 3.008, 'total_flos': 4.22072299938816e+16, 'train_loss': 1.7786762446938398, 'epoch': 5.0})

In [ ]:
print("\nPerforming evaluation...")
eval_results = trainer.evaluate()

print("\nEvaluation Results:")
for metric, value in eval_results.items():
    print(f"{metric}: {value:.4f}")



Performing evaluation...



Evaluation Results:
eval_loss: 1.7911
eval_accuracy: 0.5559
eval_Acoustic_precision: 0.0000
eval_Acoustic_recall: 0.0000
eval_Acoustic_f1: 0.0000
eval_Adult Alternative_precision: 0.0000
eval_Adult Alternative_recall: 0.0000
eval_Adult Alternative_f1: 0.0000
eval_Alternative_precision: 0.0000
eval_Alternative_recall: 0.0000
eval_Alternative_f1: 0.0000
eval_Alternative Metal_precision: 0.0000
eval_Alternative Metal_recall: 0.0000
eval_Alternative Metal_f1: 0.0000
eval_Alternative Pop_precision: 0.0000
eval_Alternative Pop_recall: 0.0000
eval_Alternative Pop_f1: 0.0000
eval_Alternative R&B_precision: 0.0000
eval_Alternative R&B_recall: 0.0000
eval_Alternative R&B_f1: 0.0000
eval_Alternative Rock_precision: 0.1176
eval_Alternative Rock_recall: 0.0952
eval_Alternative Rock_f1: 0.1053
eval_Art_precision: 0.0000
eval_Art_recall: 0.0000
eval_Art_f1: 0.0000
eval_Art Pop_precision: 0.0000
eval_Art Pop_recall: 0.0000
eval_Art Pop_f1: 0.0000
eval_Art Rock_precision: 0.0000
eval_Art Rock_recall: 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
predictions = trainer.predict(tokenized_val_dataset)
logits = predictions.predictions
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

probs = torch.softmax(torch.tensor(logits), dim=1)
confidences = probs[range(len(probs)), preds].numpy()

true_genres = [id2label[i] for i in labels]
pred_genres = [id2label[i] for i in preds]

print("\nDetailed Classification Report:")
print(classification_report(labels, preds, target_names=list(label2id.keys())))

conf_matrix = confusion_matrix(labels, preds)
print("\nConfusion Matrix:")
print(conf_matrix)

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Detailed Classification Report:
                           precision    recall  f1-score   support

                 Acoustic       0.00      0.00      0.00         4
        Adult Alternative       0.00      0.00      0.00         5
              Alternative       0.00      0.00      0.00        27
        Alternative Metal       0.00      0.00      0.00         5
          Alternative Pop       0.00      0.00      0.00        15
          Alternative R&B       0.00      0.00      0.00        13
         Alternative Rock       0.12      0.10      0.11        21
                      Art       0.00      0.00      0.00         7
                  Art Pop       0.00      0.00      0.00         4
                 Art Rock       0.00      0.00      0.00         9
                  Atlanta       0.51      0.46      0.48        72
                Australia       0.62      0.42      0.50        12
                   Ballad       0.00      0.00      0.00         8
              Baroque Pop   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
def calculate_bertscore(references, predictions):
    scorer = BERTScorer(lang="en", rescale_with_baseline=True)
    P, R, F1 = scorer.score(predictions, references)
    return torch.mean(P).item(), torch.mean(R).item(), torch.mean(F1).item()

In [ ]:
pred_genres = [id2label[pred] for pred in preds]
true_genres = [id2label[label] for label in labels]

try:
    precision, recall, f1 = calculate_bertscore(true_genres, pred_genres)
    print(f"\nBERTScore:")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1: {f1:.4f}")
except Exception as e:
    print(f"Error calculating BERTScore: {e}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



BERTScore:
Precision: 0.7653
Recall: 0.7191
F1: 0.7363


In [ ]:
model_save_path = "/content/drive/MyDrive/NLP/RoBerta/roberta_lyrics_classifier2"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"\nModel saved to {model_save_path}")


Model saved to /content/drive/MyDrive/NLP/RoBerta/roberta_lyrics_classifier2
